In [2]:
# import necessary libraries

import numpy as np
import os
import lecroy

from pathlib import Path
from scipy.fft import rfft, irfft, rfftfreq

In [44]:
parent = Path.cwd().parent  # Get the parent directory (repo basline)
data_folder = parent / 'data' / 'data-tora'  # Path of data folder
dates = os.listdir(data_folder)  # List of dates in the data folder
dates = ['091027']  # List of dates in the data folder
datasets = ['cat1','cat2']#, 'tora1']  # List of datasets to be read

def get_file_list(date, dataset):
    folder = data_folder / date / dataset
    fns = sorted(os.listdir(folder))
    fns = [ (folder / fn) for fn in fns]
    return fns

files = {dat:{ds:get_file_list(dat, ds) for ds in datasets} for dat in dates}
files

{'091027': {'cat1': [PosixPath('/Users/jansc/Documents/QIS/Spring2025/SCIQIS/Homodyne-Tomography/data/data-tora/091027/cat1/C1cat000deg00000.trc'),
   PosixPath('/Users/jansc/Documents/QIS/Spring2025/SCIQIS/Homodyne-Tomography/data/data-tora/091027/cat1/C1cat000deg00001.trc'),
   PosixPath('/Users/jansc/Documents/QIS/Spring2025/SCIQIS/Homodyne-Tomography/data/data-tora/091027/cat1/C1cat000deg00002.trc'),
   PosixPath('/Users/jansc/Documents/QIS/Spring2025/SCIQIS/Homodyne-Tomography/data/data-tora/091027/cat1/C1cat015deg00000.trc'),
   PosixPath('/Users/jansc/Documents/QIS/Spring2025/SCIQIS/Homodyne-Tomography/data/data-tora/091027/cat1/C1cat015deg00001.trc'),
   PosixPath('/Users/jansc/Documents/QIS/Spring2025/SCIQIS/Homodyne-Tomography/data/data-tora/091027/cat1/C1cat015deg00002.trc'),
   PosixPath('/Users/jansc/Documents/QIS/Spring2025/SCIQIS/Homodyne-Tomography/data/data-tora/091027/cat1/C1cat030deg00000.trc'),
   PosixPath('/Users/jansc/Documents/QIS/Spring2025/SCIQIS/Homodyne-Tomo

In [45]:
meta, times, data = lecroy.read(files[dates[0]]['cat2'][0], scale=False)
dt = meta['horiz_interval']
#fs = 1/dt
#N = len(data[0])
#t = np.linspace(0, dt*N, N, endpoint=False)
#f = rfftfreq(N, dt)
meta

{'descriptor_name': b'WAVEDESC',
 'template_name': b'LECROY_2_3',
 'comm_type': 0,
 'comm_order': 1,
 'wave_descriptor': 346,
 'user_text': 0,
 'res_desc1': 0,
 'trig_time_array': 160000,
 'ris_time_array': 0,
 'res_array1': 0,
 'wave_array1': 1270000,
 'wave_array2': 0,
 'res_array2': 0,
 'res_array3': 0,
 'instrument_name': b'LECROYWR6050A',
 'instrument_number': 14852,
 'trace_label': b'',
 'reserved1': 24816,
 'reserved2': 19,
 'wave_array_count': 1270000,
 'points_per_screen': 1250000,
 'first_valid_point': 0,
 'last_valid_point': 1269999,
 'first_point': 0,
 'sparsing_factor': 1,
 'segment_index': 0,
 'subarray_count': 10000,
 'sweeps_per_acq': 1,
 'points_per_pair': 0,
 'pair_offset': 0,
 'vertical_gain': 0.0008838911890052259,
 'vertical_offset': 0.0,
 'max_value': 90.0,
 'min_value': -91.0,
 'nominal_bits': 8,
 'nom_subarray_count': 10000,
 'horiz_interval': 3.999999886872274e-09,
 'horiz_offset': -2.503809085357032e-07,
 'pixel_offset': -2.5000000000000004e-07,
 'vert_unit': 

In [ ]:
for file in files[dates[0]]['cat1']:
    meta, times, data = lecroy.read(file, scale=False)
    dt = meta['horiz_interval']
    fs = 1/dt
    N = len(data[0])
    t = np.linspace(0, dt*N, N, endpoint=False)
    f = rfftfreq(N, dt)
    
    # Perform FFT on the collapsed data
    fft_data = rfft(data_collapsed)
    
    # Display the frequency spectrum
    print(f"Frequency spectrum for {file}:")
    print(f"Frequencies: {f}")
    print(f"FFT Data: {fft_data}")

Let's try to get a dataset for 091027/cat1 so that it has shape 10000 traces * 127 time values each * (12+1) angles + vaccum * 3 files for each angle. Thus, the shape of the array should be (10000,13,3)

In [33]:
files_cat1 = files[dates[0]]['cat1']
angles = [str(i*15) for i in range(12)] + ['vac']

#out_dir = parent / 'data' / 'processed_data'

for theta in angles:
    files_theta = [f for f in files_cat1 if 'C1cat'+f"{theta:03}" in f.name]
    data_theta = np.empty((10000, 127, 3))  # Initialize array for data
    for i,f in enumerate(files_theta):
        meta, times, data_theta[:,:,i] = lecroy.read(f, scale=False)

    data_collapsed = data_theta.mean(axis=2)  # Collapse the data by averaging over same angle
    data_collapsed = data_collapsed.mean(axis=1)  # Collapse the data by averaging over all 127 time values
    print(f"Data shape for angle {theta}: {data_collapsed.shape}")

    



Data shape for angle 0: (10000,)
Data shape for angle 15: (10000,)
Data shape for angle 30: (10000,)
Data shape for angle 45: (10000,)
Data shape for angle 60: (10000,)
Data shape for angle 75: (10000,)
Data shape for angle 90: (10000,)
Data shape for angle 105: (10000,)
Data shape for angle 120: (10000,)
Data shape for angle 135: (10000,)
Data shape for angle 150: (10000,)
Data shape for angle 165: (10000,)
Data shape for angle vac: (10000,)
